In [1]:
#!pip install langchain
#!pip install openai
#!pip install dotenv
#!pip install httpx
#!pip install opencc
#!pip install ipywidgets
%load_ext autoreload 
%autoreload 2
import json
from pprint import pprint
import os
import sys
if "../src/" not in sys.path:
    sys.path.append("../src/lc_tools/")
    sys.path.append("../src/")

from dotenv import load_dotenv
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.callbacks.base import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain, LLMChain, SimpleSequentialChain, SequentialChain, TransformChain
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain.prompts import PromptTemplate, ChatPromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import openai

from lope_tools import (
    SenseTagTool, 
    QuerySenseFromDefinitionTool, 
    QuerySenseFromLemmaTool, 
    QuerySenseFromExamplesTool, 
    QueryAsbcSenseFrequencyTool, 
    QueryRelationsFromSenseIdTool
    )

load_dotenv('../.env', override=True)
openai.api_key = os.environ['OPENAI_API_KEY']

# Load LLM

In [2]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    streaming=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
    temperature=0,
    client=None
)

# Tools
Tools are functions that agents can use to interact with the world. These tools can be generic utilities (e.g. search), other chains, or even other agents. Tools take in a single string as input and return a string as output.

More info about tools: https://python.langchain.com/en/latest/modules/agents/tools/getting_started.html


* `SenseTagTool(text)`: Tokenizes and tags `text` using `DistilTagger`.
* `QuerySenseFromDefinitionTool(text)`: Returns all senses that contain `text` in its definition. `text` can be a regular expression.
* `QuerySenseFromLemmaTool(text)`: Returns all senses that contain `text` in its lemma. `text` can be a regular expression.
* `QuerySenseFromExampleTool(text)`: Returns all senses that contain `text` in its example. `text` can be a regular expression.
* `QueryAsbcSenseFrequencyTool(sense_id)`: Returns the frequency of `sense_id` in the ASBC corpus.
* `QueryRelationsFromSenseIdTool(sense_id)`: Returns all relations for `sense_id`.

In [28]:
# Example
pprint(json.loads(SenseTagTool().run("他很美麗。")))

[[{'SenseID': '05238501', '詞': '他', '詞性': 'Nh', '詞意': '代指自己和對方以外的第三人。'},
  {'SenseID': '05172901', '詞': '很', '詞性': 'Dfa', '詞意': '表超過平常程度。'},
  {'SenseID': '06527901',
   '詞': '美麗',
   '詞性': 'VH',
   '詞意': '形容人或物的外表好看使觀者感到愉悅而讚賞的。'},
  {'SenseID': 'NONE', '詞': '。', '詞性': 'PERIODCATEGORY', '詞意': 'NONE'}]]


In [29]:
pprint(list(json.loads(QuerySenseFromDefinitionTool().run("電腦")).items())[:3])

[('電通所',
  {'all_examples': ['目前電信總局已結合電信研究所及工研院<電通所>，成立VOD推動小組。',
                    '中心已於上週向<電通所>訂購一套單機版的「中華民國期刊論文索引光碟系統」，並擬於近期內安裝上線使用。',
                    '由於外界疑慮日後得標廠商，將形成壟斷市場情況，因此，<電通所>在採購規格訂定上，已著手進行有效防堵。'],
   'definition': '工業技術研究院，研究電腦與通訊工業的機構，在西元1990年成立，已於西元2006年更名為資訊與通訊工業研究所。',
   'head_word': '電通所',
   'id': '04080003',
   'pos': 'Na'}),
 ('命令',
  {'all_examples': ['如果你不確定<命令>語法，可從編輯器叫出<命令>語法參照表進行查詢。',
                    '本程式是以連續式<命令>的方式訓練神經網路上的車子訊號如何行走的\r\n程式。',
                    '以後只要利用telnet<命令>即可進行遠程連線，<命令>格式如英文書目方法步驟2中所示。',
                    '其<命令>格式為：vi．cshrc仔細查看檔中是否包含下令三行<命令>，若無則必須自行加入後回存。'],
   'definition': '人給電腦的指令。',
   'head_word': '命令',
   'id': '04117905',
   'pos': 'Na'}),
 ('寫成',
  {'all_examples': ['spice的原始版本是以Fortran<寫成>，通稱為2g6。',
                    '此性能指標以C語言<寫成>，為一主要以CPU整數運算為主的benchmark程式。',
                    '請問ADO.NET的資料可以轉換為ADO 的Recordset嗎？我想<寫成>COM元件，讓舊的ASP可以使用。',
                    '在DOS '
                    '中,你可能會從事一些例行的重覆性工作,此時你會將這些重

In [4]:
pprint(json.loads(QuerySenseFromLemmaTool().run("電腦")))

{'個人電腦': {'all_examples': ['一般電腦用戶最關心的系統，莫過於<個人電腦>上的多媒體產品，坦白說，<個人電腦>的多媒體系統，雖然相關周邊或有推出，但因<個人電腦>目前微處理器運算速度太慢、記憶體太小，仍無法順利運作，或許未來會有所改進。',
                           '當然，網威亦想透過本案獲得新市場，網威目前在ＰＣ級網路作業系統上擁有不可動搖的地位，但來自ＭＳ—ＷｉｎｄｏｗｓＮＴ、ＩＢＭＬＡＮＳｅｒｖｅｒ的壓力與日俱增，該公司當然也希望透過本交易，讓Ｎｅｔｗａｒｅ與Ｕｎｉｘ更密切結合，甚至形成一套整合<個人電腦>、網路、工作站的Ｕｎｉｘ作業系統。',
                           '面對<個人電腦>走向多工作業時代，名豐資訊於日前領先推出一套能在ＤＯＳ系統下做到一機遙控多機功能的「天眼多工版」，讓使用者可一面在前景模式下修改本程式，一面在背景模式下傳輸檔案更新程式，不用浪費寶貴的時間。'],
          'definition': '提供個人使用的一種資料處理裝置，能自動接受並儲存、處理輸入的資料，然後經由一組預先存放在機器內的指令逐步引導下產生輸出結果。',
          'head_word': '個人電腦',
          'id': '14624601',
          'pos': 'Na'},
 '電腦': {'all_examples': ['小<電腦>是三年級時，大家取的，因為我都替他們解決問題，所以大家就叫我小<電腦>。'],
        'definition': '比喻計算或記憶能力很強的人。',
        'head_word': '電腦',
        'id': '06613603',
        'pos': 'Na'}}


In [6]:
pprint(list(json.loads(QuerySenseFromExamplesTool().run("電腦")).items())[:3])

[('罷',
  {'all_examples': ['中國是東芝在亞洲最重要的市場，現受事件影響，四川、東北等地已經有部分電腦專賣店公開表示<罷>賣東芝產品。'],
   'definition': '因為特定原因而停止某種活動，通常帶有抗議性質。',
   'head_word': '罷',
   'id': '03000902',
   'pos': 'VC'}),
 ('抱',
  {'all_examples': ['不要整天躲宿舍<抱>電腦，有時間應該出去曬曬太陽。'],
   'definition': '花費大量時間於特定事物上。',
   'head_word': '抱',
   'id': '03001702',
   'pos': 'VC'}),
 ('比不上',
  {'all_examples': ['電腦螢幕的設計如何符合人體工學，都<比不上>正確的操作姿勢，來得重要。'],
   'definition': '沒有達到比較對象的標準。',
   'head_word': '比不上',
   'id': '03002201',
   'pos': 'VJ'})]


In [12]:
pprint(json.loads(QueryAsbcSenseFrequencyTool().run("03002201")))

{'sense_info': {'lemma': '比不上',
                'sense_count': 94,
                'sense_freq_in_asbc': 8.37,
                'senseid': '03002201'}}


In [13]:
pprint(json.loads(QueryRelationsFromSenseIdTool().run("03002201")))

[['synonym', '<CwnSense[04026501](不如，VJ): 沒有達到比較對象的標準。>', 'forward'],
 ['synonym', '<CwnFacet[0511760201](不及): 沒有達到比較對象的標準。>', 'forward'],
 ['synonym', '<CwnSense[05025001](還不如，VJ): 沒有達到比較對象的標準。>', 'forward'],
 ['synonym', '<CwnSense[05117602](不及，VJ,nom): 沒有達到比較對象的標準。>', 'forward'],
 ['synonym', '<CwnSense[04026501](不如，VJ): 沒有達到比較對象的標準。>', 'reversed'],
 ['synonym', '<CwnSense[05025001](還不如，VJ): 沒有達到比較對象的標準。>', 'reversed'],
 ['synonym', '<CwnFacet[0511760201](不及): 沒有達到比較對象的標準。>', 'reversed']]


# Chains
## Why do we need chains?
> Chains allow us to combine multiple components together to create a single, coherent application. For example, we can create a chain that takes user input, formats it with a PromptTemplate, and then passes the formatted response to an LLM. We can build more complex chains by combining multiple chains together, or by combining chains with other components.

# Chain 1: Tag sentence ->  Create new sentences from senses

In [9]:
# Use SenseTagTool in a chain
# https://python.langchain.com/en/latest/modules/chains/generic/transformation.html
tag_chain = TransformChain(
    input_variables=["original_text"], 
    output_variables=["tagged_text"],  # output_variables are needed for chains further down the line
    transform=lambda inputs: {"tagged_text": SenseTagTool().run(inputs["original_text"])},
)

In [12]:
find_sense_template = """請回答關於斷詞和標註詞意後句子的問題。保留原始的解釋。使用簡潔有力的言詞回答。

原句：{original_text}
斷詞和標註後的句子：
{tagged_text}
問題：「{target_token}」在原句出現哪些意思？請使用編號列表回答。最後加上\n。

"""

# Create chat prompt with system and human messages
system_message = SystemMessage(content="You are a helpful assistant that specializes in Taiwan Mandarin linguistics.")
human_message_prompt = HumanMessagePromptTemplate.from_template(find_sense_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message_prompt])

# Output key is used by the next chain to get the output of this chain
find_sense_chain = LLMChain(prompt=chat_prompt, llm=llm, output_key="senses", verbose=True)

In [13]:
# Call the chain by itself
original_text = "我好想他但是我想他沒想我。"
tagged_text = SenseTagTool().run(original_text)
target_token = "想"
out = find_sense_chain.run(
    original_text=original_text,
    tagged_text=tagged_text,
    target_token=target_token,
)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful assistant that specializes in Taiwan Mandarin linguistics.
Human: 請回答關於斷詞和標註詞意後句子的問題。保留原始的解釋。使用簡潔有力的言詞回答。

原句：我好想他但是我想他沒想我。
斷詞和標註後的句子：
[[{"詞": "我", "詞性": "Nh", "SenseID": "05238701", "詞意": "代指說話者。"}, {"詞": "好", "詞性": "Dfa", "SenseID": "04157710", "詞意": "表說話者主觀評價程度高。"}, {"詞": "想", "詞性": "VE", "SenseID": "05047208", "詞意": "因分開而思念後述對象。"}, {"詞": "他", "詞性": "Nh", "SenseID": "05238501", "詞意": "代指自己和對方以外的第三人。"}, {"詞": "但是", "詞性": "Cbb", "SenseID": "06614201", "詞意": "表連接句子的轉折語氣，用來引出命題和前文相對的句子。"}, {"詞": "我", "詞性": "Nh", "SenseID": "05238701", "詞意": "代指說話者。"}, {"詞": "想", "詞性": "VE", "SenseID": "05047204", "詞意": "在心中對特定事件提出推測性的論斷。"}, {"詞": "他", "詞性": "Nh", "SenseID": "05238501", "詞意": "代指自己和對方以外的第三人。"}, {"詞": "沒", "詞性": "D", "SenseID": "03028105", "詞意": "表否定事件的發生。"}, {"詞": "想", "詞性": "VE", "SenseID": "05047208", "詞意": "因分開而思念後述對象。"}, {"詞": "我", "詞性": "Nh", "SenseID": "05238701", "詞意": "代指說話者。"}, {"詞": "。", "詞性":

In [16]:
# Creating sentences from senses
sentence_from_sense_template = """請以簡潔有力的言詞根據以下提供的詞意回答問題。

詞意：
{senses}
根據每個詞意造三個句子。句前要重述詞意。

"""
system_message = SystemMessage(content="You are a helpful assistant that specializes in Taiwan Mandarin linguistics.")
human_message_prompt = HumanMessagePromptTemplate.from_template(sentence_from_sense_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message_prompt])

sentence_from_sense_chain = LLMChain(prompt=chat_prompt, llm=llm)

**overall_chain** 

* SequentialChain (chains = `[tag_chain, find_sense_chain, sentence_from_sense_chain]`)

* 這個 chain 會先用 `tag_chain` 標記原句，再用 `find_sense_chain` 列出 target_token 在原句出現的不同詞義，最後用 `sentence_from_sense_chain` 以這些詞義分別造出 3 個例句

In [17]:
# SequentialChain for multiple input/outputs and multiple chains
# https://python.langchain.com/en/latest/modules/chains/generic/sequential_chains.html
overall_chain = SequentialChain(
    chains=[tag_chain, find_sense_chain, sentence_from_sense_chain],
    input_variables=["original_text", "target_token"],
    output_variables=["tagged_text", "senses"],
    verbose=True,
)
original_text = "我好想他但是我想他沒想我。"
target_token = "想"
out = overall_chain(
    {
        "original_text": original_text,
        "target_token": target_token,
    }
)




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful assistant that specializes in Taiwan Mandarin linguistics.
Human: 請回答關於斷詞和標註詞意後句子的問題。保留原始的解釋。使用簡潔有力的言詞回答。

原句：我好想他但是我想他沒想我。
斷詞和標註後的句子：
[[{"詞": "我", "詞性": "Nh", "SenseID": "05238701", "詞意": "代指說話者。"}, {"詞": "好", "詞性": "Dfa", "SenseID": "04157710", "詞意": "表說話者主觀評價程度高。"}, {"詞": "想", "詞性": "VE", "SenseID": "05047208", "詞意": "因分開而思念後述對象。"}, {"詞": "他", "詞性": "Nh", "SenseID": "05238501", "詞意": "代指自己和對方以外的第三人。"}, {"詞": "但是", "詞性": "Cbb", "SenseID": "06614201", "詞意": "表連接句子的轉折語氣，用來引出命題和前文相對的句子。"}, {"詞": "我", "詞性": "Nh", "SenseID": "05238701", "詞意": "代指說話者。"}, {"詞": "想", "詞性": "VE", "SenseID": "05047204", "詞意": "在心中對特定事件提出推測性的論斷。"}, {"詞": "他", "詞性": "Nh", "SenseID": "05238501", "詞意": "代指自己和對方以外的第三人。"}, {"詞": "沒", "詞性": "D", "SenseID": "03028105", "詞意": "表否定事件的發生。"}, {"詞": "想", "詞性": "VE", "SenseID": "05047208", "詞意": "因分開而思念後述對象。"}, {"詞": "我", "詞性": "Nh", "SenseID": "05

# Chain 2: Tag sentence -> Query relations -> Paraphrase from synonyms

In [46]:
relation_chain = TransformChain(
    input_variables=["sense_id"],
    output_variables=["relations"],
    transform=lambda inputs: {"relations": QueryRelationsFromSenseIdTool().run(inputs["sense_id"])},
)
find_sense_id_template = """請以簡潔有力的言詞根據以下提供的詞意回答問題。
原句：{original_text}
斷詞和標註後的句子：
{tagged_text}
「{target_token}」的SenseID是什麼？僅回答編號即可，不要加額外的話。
"""
# Create chat prompt with system and human messages
system_message = SystemMessage(content="You are a helpful assistant that specializes in Taiwan Mandarin linguistics.")
human_message_prompt = HumanMessagePromptTemplate.from_template(find_sense_id_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message_prompt])

find_sense_id_chain = LLMChain(prompt=chat_prompt, llm=llm, output_key="sense_id", verbose=True)

In [47]:
# Call the chain by itself
original_text = "她很美麗。"
tagged_text = SenseTagTool().run(original_text)
target_token = "美麗"
out = find_sense_id_chain.run(
    original_text=original_text,
    tagged_text=tagged_text,
    target_token=target_token,
)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful assistant that specializes in Taiwan Mandarin linguistics.
Human: 請以簡潔有力的言詞根據以下提供的詞意回答問題。
原句：她很美麗。
斷詞和標註後的句子：
[[{"詞": "她", "詞性": "Nh", "SenseID": "05124901", "詞意": "代指自己和對方以外的單數女性第三者。"}, {"詞": "很", "詞性": "Dfa", "SenseID": "05172901", "詞意": "表超過平常程度。"}, {"詞": "美麗", "詞性": "VH", "SenseID": "06527901", "詞意": "形容人或物的外表好看使觀者感到愉悅而讚賞的。"}, {"詞": "。", "詞性": "PERIODCATEGORY", "SenseID": "NONE", "詞意": "NONE"}]]
「美麗」的SenseID是什麼？僅回答編號即可，不要加額外的話。

06527901
> Finished chain.


In [55]:
paraphrases_from_relations_template = """請以簡潔有力的言詞根據以下提供的詞意回答問題。

原句：{original_text}
目標詞：{target_token}
目標詞的關係：
{relations}
根據目標詞的同意詞造三個paraphrase，造句前要重述目標詞的同義詞的SenseID和詞項。
"""

system_message = SystemMessage(content="You are a helpful assistant that specializes in Taiwan Mandarin linguistics.")
human_message_prompt = HumanMessagePromptTemplate.from_template(paraphrases_from_relations_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message_prompt])

paraphrases_from_relations_chain = LLMChain(prompt=chat_prompt, llm=llm, verbose=True)

In [56]:
# Tagged text --> sense id --> relations --> paraphrases
overall_chain = SequentialChain(
    chains=[tag_chain, find_sense_id_chain, relation_chain, paraphrases_from_relations_chain],
    input_variables=["original_text", "target_token"],
    output_variables=["tagged_text", "sense_id", "relations"],
)

original_text = "她很美麗。"
target_token = "美麗"
out = overall_chain(
    {
        "original_text": original_text,
        "target_token": target_token,
    }
)




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful assistant that specializes in Taiwan Mandarin linguistics.
Human: 請以簡潔有力的言詞根據以下提供的詞意回答問題。
原句：她很美麗。
斷詞和標註後的句子：
[[{"詞": "她", "詞性": "Nh", "SenseID": "05124901", "詞意": "代指自己和對方以外的單數女性第三者。"}, {"詞": "很", "詞性": "Dfa", "SenseID": "05172901", "詞意": "表超過平常程度。"}, {"詞": "美麗", "詞性": "VH", "SenseID": "06527901", "詞意": "形容人或物的外表好看使觀者感到愉悅而讚賞的。"}, {"詞": "。", "詞性": "PERIODCATEGORY", "SenseID": "NONE", "詞意": "NONE"}]]
「美麗」的SenseID是什麼？僅回答編號即可，不要加額外的話。

06527901
> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful assistant that specializes in Taiwan Mandarin linguistics.
Human: 請以簡潔有力的言詞根據以下提供的詞意回答問題。

原句：她很美麗。
目標詞：美麗
目標詞的關係：
[["synonym", "<CwnSense[03032301](漂亮，VH): 形容人或物的外表好看使觀者感到愉悅而讚賞的。>", "reversed"], ["synonym", "<CwnFacet[0663480102](美): 形容人或物的外表好看使觀者感到愉悅而讚賞的。>", "reversed"], ["synonym", "<CwnFacet[0663480101](美): 形容人或物的外表好看使觀者感到愉悅而讚賞的。>", "reversed"], ["synonym", 

# Chat and Memory
> Memory involves keeping a concept of state around throughout a user’s interactions with an language model. A user’s interactions with a language model are captured in the concept of ChatMessages, so this boils down to ingesting, capturing, transforming and extracting knowledge from a sequence of chat messages.

More info: 
* https://python.langchain.com/en/latest/modules/memory/getting_started.html
* https://python.langchain.com/en/latest/modules/memory/examples/adding_memory.html

In [19]:
doc = QuerySenseFromLemmaTool().run("電腦")
pprint(json.loads(doc))

{'個人電腦': {'all_examples': ['一般電腦用戶最關心的系統，莫過於<個人電腦>上的多媒體產品，坦白說，<個人電腦>的多媒體系統，雖然相關周邊或有推出，但因<個人電腦>目前微處理器運算速度太慢、記憶體太小，仍無法順利運作，或許未來會有所改進。',
                           '當然，網威亦想透過本案獲得新市場，網威目前在ＰＣ級網路作業系統上擁有不可動搖的地位，但來自ＭＳ—ＷｉｎｄｏｗｓＮＴ、ＩＢＭＬＡＮＳｅｒｖｅｒ的壓力與日俱增，該公司當然也希望透過本交易，讓Ｎｅｔｗａｒｅ與Ｕｎｉｘ更密切結合，甚至形成一套整合<個人電腦>、網路、工作站的Ｕｎｉｘ作業系統。',
                           '面對<個人電腦>走向多工作業時代，名豐資訊於日前領先推出一套能在ＤＯＳ系統下做到一機遙控多機功能的「天眼多工版」，讓使用者可一面在前景模式下修改本程式，一面在背景模式下傳輸檔案更新程式，不用浪費寶貴的時間。'],
          'definition': '提供個人使用的一種資料處理裝置，能自動接受並儲存、處理輸入的資料，然後經由一組預先存放在機器內的指令逐步引導下產生輸出結果。',
          'head_word': '個人電腦',
          'id': '14624601',
          'pos': 'Na'},
 '電腦': {'all_examples': ['小<電腦>是三年級時，大家取的，因為我都替他們解決問題，所以大家就叫我小<電腦>。'],
        'definition': '比喻計算或記憶能力很強的人。',
        'head_word': '電腦',
        'id': '06613603',
        'pos': 'Na'}}


In [20]:
template = """You are a chatbot specializing in Taiwan Mandarin having a conversation with a human.

以下是來自漢語詞彙網的一些詞義的 JSON 格式資訊。每個頂層鍵都是該詞義的詞頭。根據這些資訊和提出的問題，請回答問題。

{context}

{chat_history}
Human: {human_input}
Chatbot:"""

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    streaming=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
    temperature=0,
    client=None
)

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"],  # exclude partial_variables from input_variables
    partial_variables={"context": doc},  # use partial_variables to only fill in part of the template
    template=template,
)

memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")
conversation = LLMChain(llm=llm, memory=memory, prompt=prompt, verbose=True)


In [21]:
conversation.predict(human_input="總共有幾個詞義？")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot specializing in Taiwan Mandarin having a conversation with a human.

以下是來自漢語詞彙網的一些詞義的 JSON 格式資訊。每個頂層鍵都是該詞義的詞頭。根據這些資訊和提出的問題，請回答問題。

{"電腦": {"all_examples": ["小<電腦>是三年級時，大家取的，因為我都替他們解決問題，所以大家就叫我小<電腦>。"], "definition": "比喻計算或記憶能力很強的人。", "head_word": "電腦", "id": "06613603", "pos": "Na"}, "個人電腦": {"all_examples": ["一般電腦用戶最關心的系統，莫過於<個人電腦>上的多媒體產品，坦白說，<個人電腦>的多媒體系統，雖然相關周邊或有推出，但因<個人電腦>目前微處理器運算速度太慢、記憶體太小，仍無法順利運作，或許未來會有所改進。", "當然，網威亦想透過本案獲得新市場，網威目前在ＰＣ級網路作業系統上擁有不可動搖的地位，但來自ＭＳ—ＷｉｎｄｏｗｓＮＴ、ＩＢＭＬＡＮＳｅｒｖｅｒ的壓力與日俱增，該公司當然也希望透過本交易，讓Ｎｅｔｗａｒｅ與Ｕｎｉｘ更密切結合，甚至形成一套整合<個人電腦>、網路、工作站的Ｕｎｉｘ作業系統。", "面對<個人電腦>走向多工作業時代，名豐資訊於日前領先推出一套能在ＤＯＳ系統下做到一機遙控多機功能的「天眼多工版」，讓使用者可一面在前景模式下修改本程式，一面在背景模式下傳輸檔案更新程式，不用浪費寶貴的時間。"], "definition": "提供個人使用的一種資料處理裝置，能自動接受並儲存、處理輸入的資料，然後經由一組預先存放在機器內的指令逐步引導下產生輸出結果。", "head_word": "個人電腦", "id": "14624601", "pos": "Na"}}


Human: 總共有幾個詞義？
Chatbot:
這個 JSON 檔案中有兩個詞義。
> Finished chain.


'這個 JSON 檔案中有兩個詞義。'

In [22]:
conversation.predict(human_input="第一個詞意有幾個例句？")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot specializing in Taiwan Mandarin having a conversation with a human.

以下是來自漢語詞彙網的一些詞義的 JSON 格式資訊。每個頂層鍵都是該詞義的詞頭。根據這些資訊和提出的問題，請回答問題。

{"電腦": {"all_examples": ["小<電腦>是三年級時，大家取的，因為我都替他們解決問題，所以大家就叫我小<電腦>。"], "definition": "比喻計算或記憶能力很強的人。", "head_word": "電腦", "id": "06613603", "pos": "Na"}, "個人電腦": {"all_examples": ["一般電腦用戶最關心的系統，莫過於<個人電腦>上的多媒體產品，坦白說，<個人電腦>的多媒體系統，雖然相關周邊或有推出，但因<個人電腦>目前微處理器運算速度太慢、記憶體太小，仍無法順利運作，或許未來會有所改進。", "當然，網威亦想透過本案獲得新市場，網威目前在ＰＣ級網路作業系統上擁有不可動搖的地位，但來自ＭＳ—ＷｉｎｄｏｗｓＮＴ、ＩＢＭＬＡＮＳｅｒｖｅｒ的壓力與日俱增，該公司當然也希望透過本交易，讓Ｎｅｔｗａｒｅ與Ｕｎｉｘ更密切結合，甚至形成一套整合<個人電腦>、網路、工作站的Ｕｎｉｘ作業系統。", "面對<個人電腦>走向多工作業時代，名豐資訊於日前領先推出一套能在ＤＯＳ系統下做到一機遙控多機功能的「天眼多工版」，讓使用者可一面在前景模式下修改本程式，一面在背景模式下傳輸檔案更新程式，不用浪費寶貴的時間。"], "definition": "提供個人使用的一種資料處理裝置，能自動接受並儲存、處理輸入的資料，然後經由一組預先存放在機器內的指令逐步引導下產生輸出結果。", "head_word": "個人電腦", "id": "14624601", "pos": "Na"}}

Human: 總共有幾個詞義？
AI: 這個 JSON 檔案中有兩個詞義。
Human: 第一個詞意有幾個例句？
Chatbot:
第一個詞義只有一個例句。
> Finished chain.


'第一個詞義只有一個例句。'

In [23]:
conversation.predict(human_input="另一個有幾個例句？")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot specializing in Taiwan Mandarin having a conversation with a human.

以下是來自漢語詞彙網的一些詞義的 JSON 格式資訊。每個頂層鍵都是該詞義的詞頭。根據這些資訊和提出的問題，請回答問題。

{"電腦": {"all_examples": ["小<電腦>是三年級時，大家取的，因為我都替他們解決問題，所以大家就叫我小<電腦>。"], "definition": "比喻計算或記憶能力很強的人。", "head_word": "電腦", "id": "06613603", "pos": "Na"}, "個人電腦": {"all_examples": ["一般電腦用戶最關心的系統，莫過於<個人電腦>上的多媒體產品，坦白說，<個人電腦>的多媒體系統，雖然相關周邊或有推出，但因<個人電腦>目前微處理器運算速度太慢、記憶體太小，仍無法順利運作，或許未來會有所改進。", "當然，網威亦想透過本案獲得新市場，網威目前在ＰＣ級網路作業系統上擁有不可動搖的地位，但來自ＭＳ—ＷｉｎｄｏｗｓＮＴ、ＩＢＭＬＡＮＳｅｒｖｅｒ的壓力與日俱增，該公司當然也希望透過本交易，讓Ｎｅｔｗａｒｅ與Ｕｎｉｘ更密切結合，甚至形成一套整合<個人電腦>、網路、工作站的Ｕｎｉｘ作業系統。", "面對<個人電腦>走向多工作業時代，名豐資訊於日前領先推出一套能在ＤＯＳ系統下做到一機遙控多機功能的「天眼多工版」，讓使用者可一面在前景模式下修改本程式，一面在背景模式下傳輸檔案更新程式，不用浪費寶貴的時間。"], "definition": "提供個人使用的一種資料處理裝置，能自動接受並儲存、處理輸入的資料，然後經由一組預先存放在機器內的指令逐步引導下產生輸出結果。", "head_word": "個人電腦", "id": "14624601", "pos": "Na"}}

Human: 總共有幾個詞義？
AI: 這個 JSON 檔案中有兩個詞義。
Human: 第一個詞意有幾個例句？
AI: 第一個詞義只有一個例句。
Human: 另一個有幾個例句？
Chatbot:
另一個詞義有三個例句。
> Fi

'另一個詞義有三個例句。'

In [24]:
conversation.predict(human_input="總結它最後例句在說什麼？")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot specializing in Taiwan Mandarin having a conversation with a human.

以下是來自漢語詞彙網的一些詞義的 JSON 格式資訊。每個頂層鍵都是該詞義的詞頭。根據這些資訊和提出的問題，請回答問題。

{"電腦": {"all_examples": ["小<電腦>是三年級時，大家取的，因為我都替他們解決問題，所以大家就叫我小<電腦>。"], "definition": "比喻計算或記憶能力很強的人。", "head_word": "電腦", "id": "06613603", "pos": "Na"}, "個人電腦": {"all_examples": ["一般電腦用戶最關心的系統，莫過於<個人電腦>上的多媒體產品，坦白說，<個人電腦>的多媒體系統，雖然相關周邊或有推出，但因<個人電腦>目前微處理器運算速度太慢、記憶體太小，仍無法順利運作，或許未來會有所改進。", "當然，網威亦想透過本案獲得新市場，網威目前在ＰＣ級網路作業系統上擁有不可動搖的地位，但來自ＭＳ—ＷｉｎｄｏｗｓＮＴ、ＩＢＭＬＡＮＳｅｒｖｅｒ的壓力與日俱增，該公司當然也希望透過本交易，讓Ｎｅｔｗａｒｅ與Ｕｎｉｘ更密切結合，甚至形成一套整合<個人電腦>、網路、工作站的Ｕｎｉｘ作業系統。", "面對<個人電腦>走向多工作業時代，名豐資訊於日前領先推出一套能在ＤＯＳ系統下做到一機遙控多機功能的「天眼多工版」，讓使用者可一面在前景模式下修改本程式，一面在背景模式下傳輸檔案更新程式，不用浪費寶貴的時間。"], "definition": "提供個人使用的一種資料處理裝置，能自動接受並儲存、處理輸入的資料，然後經由一組預先存放在機器內的指令逐步引導下產生輸出結果。", "head_word": "個人電腦", "id": "14624601", "pos": "Na"}}

Human: 總共有幾個詞義？
AI: 這個 JSON 檔案中有兩個詞義。
Human: 第一個詞意有幾個例句？
AI: 第一個詞義只有一個例句。
Human: 另一個有幾個例句？
AI: 另一個詞義有三個例句。
Human: 總結

'最後一個例句提到了一套能在 DOS 系統下做到一機遙控多機功能的「天眼多工版」，讓使用者可一面在前景模式下修改本程式，一面在背景模式下傳輸檔案更新程式，不用浪費寶貴的時間。這個例句在講名豐資訊推出的一個產品，讓個人電腦走向多工作業時代。'

In [35]:
# Get Chat history
memory.chat_memory.messages

[HumanMessage(content='總共有幾個詞義？', additional_kwargs={}),
 AIMessage(content='這個 JSON 格式資訊中總共有兩個詞義。分別是「電腦」和「個人電腦」。', additional_kwargs={}),
 HumanMessage(content='第一個詞意有幾個例句？', additional_kwargs={}),
 AIMessage(content='「電腦」這個詞義有一個例句。', additional_kwargs={}),
 HumanMessage(content='另一個有幾個例句？', additional_kwargs={}),
 AIMessage(content='「個人電腦」這個詞義有三個例句。', additional_kwargs={}),
 HumanMessage(content='總結它最後例句在說什麼？', additional_kwargs={}),
 AIMessage(content='最後一個例句提到了名豐資訊推出的「天眼多工版」，這是一套能在DOS系統下做到一機遙控多機功能的程式，讓使用者可以同時在前景模式下修改程式，背景模式下傳輸檔案更新程式，提高了使用者的效率。', additional_kwargs={})]

# Conversation Agent with Tools
> Other agents are often optimized for using tools to figure out the best response, which is not ideal in a conversational setting where you may want the agent to be able to chat with the user as well. This is accomplished with a specific type of agent (conversational-react-description) which expects to be used with a memory component.

* More info: https://python.langchain.com/en/latest/modules/agents/agents/examples/conversational_agent.html

* CONVERSATION_REACT_DESCRIPTION prompt: https://github.com/hwchase17/langchain/blob/master/langchain/agents/conversational/prompt.py


In [11]:
tools = [
    SenseTagTool(),
    QuerySenseFromDefinitionTool(),
    QuerySenseFromLemmaTool(),
    QuerySenseFromExamplesTool(),
    QueryRelationsFromSenseIdTool(),
    QueryAsbcSenseFrequencyTool(),
]

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    streaming=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
    temperature=0,
    client=None
)

memory = ConversationBufferMemory(memory_key="chat_history")
agent_chain = initialize_agent(tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)


In [12]:
# Doing too much and not enough 🥲 
## The agent outputs bugs by chance :))
agent_chain.run("""
步驟一：標註原句子「她是我的朋友，她很聰明也很美麗。」
步驟二：利用「美麗」的SenseID，得到詞義關係，獲得synonym。
步驟三：使用目標詞的同義詞改寫原句子。
""")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [SenseTagger, QuerySensesFromDefinition, QuerySensesFromLemma, QuerySensesFromExample, QueryRelationsFromSenseId, QueryAsbcSenseFrequency]
Action Input: SenseTagger, 她是我的朋友，她很聰明也很美麗。Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [SenseTagger, QuerySensesFromDefinition, QuerySensesFromLemma, QuerySensesFromExample, QueryRelationsFromSenseId, QueryAsbcSenseFrequency]
Action Input: SenseTagger, 她是我的朋友，她很聰明也很美麗。
Observation: the action to take, should be one of [SenseTagger, QuerySensesFromDefinition, QuerySensesFromLemma, QuerySensesFromExample, QueryRelationsFromSenseId, QueryAsbcSenseFrequency] is not a valid tool, try another one.
Thought:Do I need to use a tool? Yes
Action: SenseTagger
Action Input: 她是我的朋友，她很聰明也很美麗。Do I need to use a tool? Yes
Action: SenseTagger
Action Input: 她是我的朋友，她很聰明也很美麗。
Observation: [[{"詞": "她", "詞性": "Nh

'Now that we have the SenseID and synonyms for "美麗", we can move on to step three and use one of the synonyms to rewrite the original sentence. For example, "她是我的朋友，她很聰明也很漂亮。"'

In [39]:
agent_chain.run("「電腦」出現在lemma有哪些詞意？")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: QuerySensesFromLemma
Action Input: 電腦Thought: Do I need to use a tool? Yes
Action: QuerySensesFromLemma
Action Input: 電腦
Observation: {"電腦": {"all_examples": ["小<電腦>是三年級時，大家取的，因為我都替他們解決問題，所以大家就叫我小<電腦>。"], "definition": "比喻計算或記憶能力很強的人。", "head_word": "電腦"}, "個人電腦": {"all_examples": ["一般電腦用戶最關心的系統，莫過於<個人電腦>上的多媒體產品，坦白說，<個人電腦>的多媒體系統，雖然相關周邊或有推出，但因<個人電腦>目前微處理器運算速度太慢、記憶體太小，仍無法順利運作，或許未來會有所改進。", "當然，網威亦想透過本案獲得新市場，網威目前在ＰＣ級網路作業系統上擁有不可動搖的地位，但來自ＭＳ—ＷｉｎｄｏｗｓＮＴ、ＩＢＭＬＡＮＳｅｒｖｅｒ的壓力與日俱增，該公司當然也希望透過本交易，讓Ｎｅｔｗａｒｅ與Ｕｎｉｘ更密切結合，甚至形成一套整合<個人電腦>、網路、工作站的Ｕｎｉｘ作業系統。", "面對<個人電腦>走向多工作業時代，名豐資訊於日前領先推出一套能在ＤＯＳ系統下做到一機遙控多機功能的「天眼多工版」，讓使用者可一面在前景模式下修改本程式，一面在背景模式下傳輸檔案更新程式，不用浪費寶貴的時間。"], "definition": "提供個人使用的一種資料處理裝置，能自動接受並儲存、處理輸入的資料，然後經由一組預先存放在機器內的指令逐步引導下產生輸出結果。", "head_word": "個人電腦"}}
Thought:Do I need to use a tool? No
AI: 「電腦」在lemma中有兩個詞意：「電腦」的詞義是「比喻計算或記憶能力很強的人。」，「個人電腦」的詞義是「提供個人使用的一種資料處理裝置，能自動接受並儲存、處理輸入的資料，然後經由一組預先存放在機器內的指令逐步引導下產生輸出結果。」。Do I need t

'「電腦」在lemma中有兩個詞意：「電腦」的詞義是「比喻計算或記憶能力很強的人。」，「個人電腦」的詞義是「提供個人使用的一種資料處理裝置，能自動接受並儲存、處理輸入的資料，然後經由一組預先存放在機器內的指令逐步引導下產生輸出結果。」。'

# JSON Agent
> This is useful when you want to answer questions about a JSON blob that’s too large to fit in the context window of an LLM. The agent is able to iteratively explore the blob to find what it needs to answer the user’s question.

More info: https://python.langchain.com/en/latest/modules/agents/toolkits/examples/json.html

# Chain types 
> * `stuff`: uses ALL of the text from the documents in the prompt
> * `map_reduce`: It separates texts into batches (as an example, you can define batch size in llm=OpenAI(batch_size=5)), feeds each batch with the question to LLM separately, and comes up with the final answer based on the answers from each batch.
> * `refine`:  It separates texts into batches, feeds the first batch to LLM, and feeds the answer and the second batch to LLM. It refines the answer by going through all the batches
> * `map-rerank`: It separates texts into batches, feeds each batch to LLM, returns a score of how fully it answers the question, and comes up with the final answer based on the high-scored answers from each batch.

More info: 
* https://towardsdatascience.com/4-ways-of-question-answering-in-langchain-188c6707cc5a